In [11]:
import torch
import torch.nn as nn
from torchvision import transforms

from einops import repeat
from einops.layers.torch import Rearrange



In [1]:
!pip install opencv-python

  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [16]:

import cv2
img_path = r"C:\Users\pshun\Documents\python\tkinter_killingtime\niwatori.jpg"
img = cv2.imread(img_path, -1)
img_size = min(img.shape[:2])

preprocess = transforms.Compose([
   transforms.ToPILImage(), # numpy.ndarray→Pillowオブジェクトに変換                                
   #transforms.Resize(img_size), # 画像サイズを256に変換
   transforms.CenterCrop(img_size), # 画像サイズ224でセンタークロッピング
   transforms.ToTensor(), # torch.Tensorオブジェクト化
   transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # 正規化
])

# OpenCVで読み込んだ画像データ(numpy.ndarray)を前処理を適用する
img_tensor = preprocess(img) # [ch, img_size, img_size]
img_tensor = img_tensor.unsqueeze(0)
print(img_tensor.shape)

torch.Size([1, 3, 180, 180])


In [35]:
class PatchEmbedding(nn.Module):
    def __init__(self, img_size, patch_size, in_channels, embed_dim):
        super().__init__()
        self.patch_size = patch_size
        self.grid_size = img_size // patch_size
        self.num_patches = self.grid_size * self.grid_size
        self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)

    def forward(self, x):
        # 入力xの形状は[batch_size, in_channels, img_size, img_size]
        x = self.proj(x)
        x = x.flatten(2)
        x = x.transpose(1, 2) # [batch_size, patch_size, embed_dim]
        return x

ps = 30
embed_dim = 5
in_channels = 3
num_patches = (img_size//ps)**2
pe = PatchEmbedding(img_size, ps, in_channels, embed_dim).forward(img_tensor)
print(pe.shape)

torch.Size([1, 36, 5])


In [38]:
class PositionEncoding(nn.Module):
    def __init__(self, embed_dim, num_patches):
        super().__init__()
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))

    def forward(self, x):
        return x + self.pos_embed
    
pos_em = PositionEncoding(embed_dim, num_patches)
print(pos_em.forward(pe).shape)

torch.Size([1, 36, 5])
torch.Size([1, 36, 5])


In [ ]:
class TransformerEncoderBlock(nn.Module)